In [1]:
%matplotlib widget

import sys  
sys.path.insert(0, '/home/cbisot/pycode/MscThesis/')
import pandas as pd
from sample.util import get_dates_datetime, get_dirname, get_plate_number, get_postion_number

import ast
from sample.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from sample.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from sample.pipeline.functions.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from sample.pipeline.functions.experiment_class_surf import Experiment
from sample.paths.directory import run_parallel, find_state, directory_scratch, directory_project, path_code

from sample.notebooks.analysis.data_info import *
import matplotlib.patches as mpatches
from statsmodels.stats import weightstats as stests


In [2]:
window=800
results={}
for treatment in treatments.keys():
    insts = treatments[treatment]
    for inst in insts:
        results[inst] = pickle.load(open(f'{path_code}/MscThesis/Results/straight_{window}_{inst}.pick', "rb"))

FileNotFoundError: [Errno 2] No such file or directory: '/home/cbisot/pycode//MscThesis/Results/straight_800_(10, 0, 10).pick'

***tortuosities vs treatment***

In [3]:
column_names = ["plate","inst", "treatment", "angle", "curvature","density","growth","speed","straightness","t","hyph"]
infos = pd.DataFrame(columns=column_names)
for treatment in treatments.keys():
    insts = treatments[treatment]
    for inst in insts:
        angles, curvatures, densities,growths,speeds,tortuosities,ts,hyphs = results[inst]
        for i,angle in enumerate(angles):
            new_line = pd.DataFrame(
                {   "plate": [plate_number[inst]],
                    "inst": [inst],
                    "treatment": [treatment],
                    "angle": [angle],
                    "curvature": [curvatures[i]],
                    "density": [densities[i]],
                    "growth": [growths[i]],
                    "speed": [speeds[i]],
                    "straightness": [tortuosities[i]],
                     "t": [ts[i]],
                     "hyph": [hyphs[i]],
                }
            )  # index 0 for
            # mothers need to be modified to resolve multi mother issue
            infos = infos.append(new_line, ignore_index=True)

KeyError: (10, 0, 10)

In [4]:
corrected = infos.loc[infos["straightness"] <= 1]

In [10]:
corrected

,plate,inst,treatment,angle,curvature,density,growth,speed,straightness,t,hyph
0,296,"(9, 0, 11)",25,6.864531,0.011940,1.973913,574.929348,143.732337,0.997008,5.316667,65
1,296,"(9, 0, 11)",25,3.628532,0.006586,1.834783,550.984831,137.746208,0.988864,9.316667,65
2,296,"(9, 0, 11)",25,-2.438101,-0.003874,2.368116,629.319466,157.329867,0.992084,13.316667,65
3,296,"(9, 0, 11)",25,3.433630,0.004524,2.665217,759.005213,189.751303,0.997847,17.316667,65
4,296,"(9, 0, 11)",25,-18.883147,-0.025417,3.871014,742.928653,185.732163,0.990694,21.316667,65
...,...,...,...,...,...,...,...,...,...,...,...
2211,351,"(35, 0, 15)",25,12.425943,0.020830,1.286957,596.533199,149.133300,0.973802,56.016667,17351
2212,351,"(35, 0, 15)",25,-6.674771,-0.012112,4.501449,551.091249,137.772812,0.974202,56.016667,17566
2213,351,"(35, 0, 15)",25,16.831669,0.033944,2.695652,495.858547,123.450261,0.993517,52.000000,18328
2214,351,"(35, 0, 15)",25,11.477627,0.024167,2.403623,474.929134,118.732283,0.989865,56.016667,18328


In [5]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
bplot1 = corrected.boxplot(column = ['speed'],by="plate",figsize =(9,8),ax =ax,patch_artist=True, showfliers=False)
colors = ['lightblue']+ ['pink'] +['lightgreen']
for i,(artist, col) in enumerate(zip(ax.artists, colors)):
    artist.set_edgecolor(col)
    artist.set_facecolor(col) 
ax.set_xlabel('Plate')
ax.set_ylabel('Speed')
ax.set_ylim(0.9)
plt.show()

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [6]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
bplot1 = corrected.boxplot(column = ['straightness'],by="plate",figsize =(9,8),ax =ax,patch_artist=True, showfliers=False)
colors = ['lightblue']+ ['pink'] +['lightgreen']
for i,(artist, col) in enumerate(zip(ax.artists, colors)):
    artist.set_edgecolor(col)
    artist.set_facecolor(col) 
ax.set_xlabel('Treatment')
ax.set_ylabel('Straightness')
ax.set_ylim(0.9)
plt.show()

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [16]:
corrected.boxplot(column = ['straightness'],by="treatment",figsize =(9,8))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [43]:
fitted_data,fitted_lambda = stats.boxcox(1-corrected.loc[corrected["treatment"] == 'baits']['straightness']) 

In [55]:
fitted_lambda 

0.053559026311838766

In [44]:
data25 = stats.boxcox(1-corrected.loc[corrected["treatment"] == '25']['straightness'],lmbda = fitted_lambda) 
data30 = stats.boxcox(1-corrected.loc[corrected["treatment"] == '30']['straightness'],lmbda = fitted_lambda) 
databaits = stats.boxcox(1-corrected.loc[corrected["treatment"] == 'baits']['straightness'],lmbda = fitted_lambda) 

In [47]:
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
# ax.hist(corrected.loc[corrected["treatment"] == 'baits']['straightness'],30)
ax.hist(databaits,30)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  7.,  1.,  8.,  7.,  8., 13.,
        28., 37., 42., 47., 37., 56., 41., 26., 18., 23., 23., 12., 17.,
         6.,  1.,  5.,  3.]),
 array([-7.5338744 , -7.33467203, -7.13546966, -6.9362673 , -6.73706493,
        -6.53786256, -6.3386602 , -6.13945783, -5.94025546, -5.7410531 ,
        -5.54185073, -5.34264836, -5.143446  , -4.94424363, -4.74504126,
        -4.5458389 , -4.34663653, -4.14743416, -3.94823179, -3.74902943,
        -3.54982706, -3.35062469, -3.15142233, -2.95221996, -2.75301759,
        -2.55381523, -2.35461286, -2.15541049, -1.95620813, -1.75700576,
        -1.55780339]),
 <a list of 30 Patch objects>)

In [19]:
import statsmodels.formula.api as sm
sm.ols('straightness ~ density + speed',data = corrected.loc[corrected["treatment"] == '30']).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           straightness   R-squared:                       0.150
Model:                            OLS   Adj. R-squared:                  0.148
Method:                 Least Squares   F-statistic:                     73.12
Date:                Fri, 29 Jan 2021   Prob (F-statistic):           5.81e-30
Time:                        12:17:55   Log-Likelihood:                 1815.1
No. Observations:                 829   AIC:                            -3624.
Df Residuals:                     826   BIC:                            -3610.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.9467      0.004    212.003      0.000       0.938       0.955
density       -0.0011      0.000     -2.729      0.006      -0.002      -0.000
speed          0.0001   1.45e-05      9.626      0.000       0.000       0.000
==============================================================================
Omnibus:                      472.734   Durbin-Watson:                   1.936
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3568.364
Skew:                          -2.550   Prob(JB):                         0.00
Kurtosis:                      11.791   Cond. No.                     1.20e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.2e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [18]:
sm.ols('straightness ~ density + speed',data = corrected.loc[corrected["plate"] == 69]).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           straightness   R-squared:                       0.100
Model:                            OLS   Adj. R-squared:                  0.096
Method:                 Least Squares   F-statistic:                     26.91
Date:                Fri, 29 Jan 2021   Prob (F-statistic):           8.31e-12
Time:                        12:16:41   Log-Likelihood:                 1220.5
No. Observations:                 487   AIC:                            -2435.
Df Residuals:                     484   BIC:                            -2422.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.9606      0.005    198.905      0.000       0.951       0.970
density       -0.0002      0.001     -0.322      0.748      -0.001       0.001
speed          0.0001   1.77e-05      6.454      0.000    7.94e-05       0.000
==============================================================================
Omnibus:                      346.831   Durbin-Watson:                   1.559
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3940.374
Skew:                          -3.081   Prob(JB):                         0.00
Kurtosis:                      15.499   Cond. No.                     1.24e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.24e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [20]:
1000/1.725

579.7101449275362

In [54]:
ztest ,pval1 = stests.ztest(data25,data30, value=0,alternative='two-sided')
pval1

3.952093842738811e-33

In [52]:
ztest ,pval1 = stests.ztest(data25,databaits, value=0,alternative='two-sided')
pval1

1.5692013454492557e-10

In [48]:
ztest ,pval1 = stests.ztest(data25,data30,corrected.loc[corrected["inst"] == (22,8,26)]['tortuosity'], corrected.loc[corrected["inst"] == (26,30,53)]['tortuosity'], value=0,alternative='two-sided')
pval1

NameError: name 'stests' is not defined

In [66]:
corrected = infos.loc[infos["speed"] >= 100]
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
bplot1 = corrected.boxplot(column = ["speed"],by="plate",figsize =(9,8),ax =ax,patch_artist=True)
colors = ['pink']*3+['lightblue']*3 +['lightgreen']*3
for i,(artist, col) in enumerate(zip(ax.artists, colors)):
    artist.set_edgecolor(col)
    artist.set_facecolor(col) 
ax.set_xlabel('Plate number')
ax.set_ylabel('Straightness')
plt.show()

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
window=800
for treatment in treatments.keys():
    insts = treatments[treatment]
    for inst in insts:
        result = get_curvature_density(inst,window)
        pickle.dump(result, open(f'{path_code}/MscThesis/Results/straight_{window}_{inst}.pick', "wb"))

begin = 2020-07-01 15:57:00 
  end = 2020-07-04 19:57:00
There is 148 RH
begin = 2020-11-23 15:01:00 
  end = 2020-11-26 03:02:00


In [42]:
angles,curvatures = get_all_curvature((24,104,116))

begin = 2020-12-08 10:57:00 
  end = 2020-12-10 11:07:00


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

There is 531 RH


In [43]:
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.set_xlabel('angle(°)')
ax.set_ylabel('density')
# ax.set_xlim((0,2))
ax.set_title(f'mean = {round(np.nanmean(angles),max(2,-int(np.log10(abs(np.nanmean(angles))))+2))}, std = {round(np.nanstd(angles),max(2,-int(np.log10(np.nanstd(angles)))+2))}')
ax.hist(angles,10,density=True)
np.nanmean(angles),np.nanstd(angles)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(-4.00015425895818, 53.04022492898531)

In [45]:
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.set_xlabel('"curvature",°/um')
ax.set_ylabel('density')
# ax.set_xlim((0,2))
ax.set_title(f'mean = {round(np.nanmean(curvatures),max(2,-int(np.log10(abs(np.nanmean(curvatures))))+2))}, std = {round(np.nanstd(curvatures),max(2,-int(np.log10(np.nanstd(curvatures)))+2))}')

ax.hist(curvatures,10,density=True)
np.nanmean(curvatures),np.nanstd(curvatures)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(-0.0056825003334708595, 0.05425367450462862)

In [67]:
def get_curvature_density(inst,window):
    exp = get_exp(inst)
    skeletons = [sparse.csr_matrix(skel) for skel in exp.skeletons]
    RH, BAS, max_growths, total_growths, lengths, branch_frequ, select_hyph = get_rh_bas(exp,criter)
    angles = []
    curvatures = []
    densities = []
    growths = []
    speeds = []
    tortuosities = []
    print(f'There is {len(RH)} RH')
    for hyph in RH:
        for i,t in enumerate(hyph.ts[:-1]):
            tp1=hyph.ts[i+1]
            segs,nodes = get_pixel_growth_and_new_children(hyph,t,tp1)
            speed = np.sum([get_length_um(seg) for seg in segs])/get_time(exp,t,tp1)
            total_growth = speed * get_time(exp,t,tp1)
            curve = [pixel for seg in segs for pixel in seg]
            pos = hyph.end.pos(t)
            x,y = pos[0],pos[1]
            straight_distance = np.linalg.norm(hyph.end.pos(t)-hyph.end.pos(tp1))
            skeleton=skeletons[t][x-window:x+window,y-window:y+window]
            density = skeleton.count_nonzero()/(window*1.725)
            curve_array = np.array(curve)
            res = 50
            index = min(res,curve_array.shape[0]-1)
            vec1 = curve_array [index]-curve_array [0]
            vec2 = curve_array [-1]-curve_array [-index-1]
            if np.linalg.norm(vec1)>0 and np.linalg.norm(vec2)>0:
                begin=vec1/np.linalg.norm(vec1)
                end=vec2/np.linalg.norm(vec2)
                dot_product = min(np.dot(begin, end),1)
                if (begin[1] * end[0] - end[1] * begin[0] >= 0):  # determinant
                    angle = -np.arccos(dot_product) / (2 * np.pi) * 360
                else:
                    angle = np.arccos(dot_product) / (2 * np.pi) * 360
                inv_tortuosity = (straight_distance*1.725)/total_growth
                if  speed>=100 and speed <400 and hyph.end.degree(tp1)<3 and inv_tortuosity>0.8 and inv_tortuosity<1.1:
                    if np.isnan((angle/total_growth)):
                        print(angle,total_growth,dot_product)
                    angles.append(angle)
                    curvatures.append(angle/total_growth)
                    densities.append(density)
                    growths.append(total_growth)
                    speeds.append(speed)
                    tortuosities.append(inv_tortuosity)
    return(angles, curvatures, densities,growths,speeds,tortuosities)

In [68]:
angles,curvatures, densities,growths, speeds, tortuosities = get_curvature_density((40,0,37),800)

begin = 2020-11-19 20:21:00 
  end = 2020-11-25 18:06:00
There is 189 RH


In [85]:
plt.close('all')
zipped_list = zip(speeds,np.abs(tortuosities))
sorted_zip = sorted(zipped_list)
densities_sort,curvatures_sort = zip(*sorted_zip)
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('speed ($\mu m.hour^{-1})$')
ax.set_ylabel('straightness ($\mu m. \mu m^{-1}$)')
xlim = int(np.max(densities_sort))+1
ax.set_xlim((0,xlim))
ax.set_ylim((0.8,1.05))
slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
x= range(0,xlim)

N=30
ax.scatter(densities_sort,curvatures_sort,color='yellow',label='observed speeds')
ax.plot(uniform_filter1d(densities_sort, size=N),uniform_filter1d(curvatures_sort, size=N),color='green',label = 'moving average')
# ax.set_title(r'$r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p_value = {round(p_value**2,max(2,-int(np.log10(p_value**2))+2))}, slope = {round(slope,4)}')
x= range(0,xlim+1)
legend_str = r'linear regression : $r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p-value = {round(p_value,max(2,-int(np.log10(p_value))+2))}, slope = {round(slope,4)}'
line, = ax.plot(x,x*slope+intercept,color='red',label = legend_str)
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [89]:
plt.close('all')
zipped_list = zip(densities,np.abs(tortuosities))
sorted_zip = sorted(zipped_list)
densities_sort,curvatures_sort = zip(*sorted_zip)
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('density ($\mu m. \mu m^{-1}$)')
ax.set_ylabel('straightness ($\mu m. \mu m^{-1}$)')
xlim = int(np.max(densities_sort))+1
ax.set_xlim((0,xlim))
ax.set_ylim((0.8,1.05))
slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
x= range(0,xlim)

N=30
ax.scatter(densities_sort,curvatures_sort,color='yellow',label='observed speeds')
ax.plot(uniform_filter1d(densities_sort, size=N),uniform_filter1d(curvatures_sort, size=N),color='green',label = 'moving average')
# ax.set_title(r'$r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p_value = {round(p_value**2,max(2,-int(np.log10(p_value**2))+2))}, slope = {round(slope,4)}')
x= range(0,xlim+1)
legend_str = r'linear regression : $r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p-value = {round(p_value,max(2,-int(np.log10(p_value))+2))}, slope = {round(slope,4)}'
line, = ax.plot(x,x*slope+intercept,color='red',label = legend_str)
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
zipped_list = zip(speeds,np.abs(tortuosities))
sorted_zip = sorted(zipped_list)
densities_sort,curvatures_sort = zip(*sorted_zip)
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
# ax.set_xlabel('time since start of imaging ($hour$)')
# ax.set_ylabel('speed ($\mu m.hour^{-1})$')
xlim = 10
# ax.set_xlim((0,xlim))
ax.set_ylim((0,3))
slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
x= range(0,xlim)

N=30
ax.scatter(densities_sort,curvatures_sort,color='yellow',label='observed speeds')
ax.plot(uniform_filter1d(densities_sort, size=N),uniform_filter1d(curvatures_sort, size=N),color='green',label = 'moving average')
# ax.set_title(r'$r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p_value = {round(p_value**2,max(2,-int(np.log10(p_value**2))+2))}, slope = {round(slope,4)}')
x= range(0,xlim+1)
legend_str = r'linear regression : $r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p-value = {round(p_value**2,max(2,-int(np.log10(p_value**2))+2))}, slope = {round(slope,4)}'
line, = ax.plot(x,x*slope+intercept,color='red',label = legend_str)
ax.legend()

In [12]:

slope, intercept, r_value, p_value, std_err = stats.linregress(curvatures,np.abs(curvatures))

In [13]:
p_value

7.875893889324231e-13

In [14]:
slope

-0.0004887144099304017

In [ ]:
exp_inst = [(22,8,40)]

experiments = []
for inst in exp_inst:
    plate = inst[0]
    begin = inst[1]
    end = inst[2]
    dates_datetime = get_dates_datetime(directory,plate)
    print('begin =',dates_datetime[begin],'\n  end =',dates_datetime[end])
    dates_datetime_chosen=dates_datetime[begin:end+1]
    dates = dates_datetime_chosen
    exp = pickle.load( open(f'{directory}/Analysis_Plate{plate}_{dates[0]}_{dates[-1]}/experiment_{plate}.pick', "rb" ) )
    experiments.append(exp)

In [ ]:
select_hyph = {}
for hyph in exp.hyphaes:
    select_hyph[hyph] = []
    for i,t in enumerate(hyph.ts[:-1]):
        tp1=hyph.ts[i+1]
        pixels,nodes = get_pixel_growth_and_new_children(hyph,t,tp1)
        length = np.sum([get_length_um(seg) for seg in pixels])/get_time(exp,t,tp1)
        select_hyph[hyph].append((t,hyph.ts[i+1],length,pixels))
        

In [ ]:
max_growths = []
total_growths = []
lengths = []
branch_frequ = []
hyph_l = []
RH=[]
BAS=[]
thresh_growth = 100
thresh_length = 2000
for hyph in exp.hyphaes:
    growths = [c[2] for c in select_hyph[hyph]]
    ts = [c[0] for c in select_hyph[hyph]]
    tp1s = [c[1] for c in select_hyph[hyph]]
    if len(growths)>0:
        length = hyph.get_length_um(hyph.ts[-1])
        nodes = hyph.get_nodes_within(hyph.ts[-1])[0]
        max_growth = np.max(growths)
        total_growth = np.sum([growth * get_time(exp,ts[i],tp1s[i]) for i,growth in enumerate(growths)])
        if max_growth>=thresh_growth and length>=thresh_length:
            RH.append(hyph)
        else:
            BAS.append(hyph)
        lengths.append(length)
        max_growths.append(max_growth)
        total_growths.append(total_growth)
        branch_frequ.append((len(nodes)-1)/(length+1))
        hyph_l.append(hyph)

In [ ]:
rh = choice(RH)
patterns = select_hyph[rh]
ts = [c[0] for c in select_hyph[rh]]
tp1s = [c[1] for c in select_hyph[rh]]
growths = [c[2] for c in select_hyph[rh]]
if len(growths)==0:
    print('error')
total_growth = np.sum([growth * get_time(exp,ts[i],tp1s[i]) for i,growth in enumerate(growths)])
curve = [pixel for info in patterns for seg in info[3] for pixel in seg]

In [ ]:
curve_array = np.array(curve)

In [ ]:
res = 50
vec1 = curve_array [res]-curve_array [0]
vec2 = curve_array [-1]-curve_array [-res-1]

In [90]:
begin=vec1/np.linalg.norm(vec1)
end=vec2/np.linalg.norm(vec2)

In [91]:
plt.close('all')
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
# ax.set_xlabel('density(um hypha/um^2)')
# ax.set_ylabel('speed (um/hour)')
# # ax.set_xlim((0,15000))
# ax.set_ylim((0,450))
ax.scatter(curve_array[:,1],curve_array[:,0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [92]:
dot_product = np.dot(begin, end)
if (begin[1] * end[0] - end[1] * begin[0] >= 0):  # determinant
    angle = -np.arccos(dot_product) / (2 * np.pi) * 360
else:
    angle = np.arccos(dot_product) / (2 * np.pi) * 360

In [93]:
angle/total_growth

-0.004606482394418124

In [94]:
angle

-11.479949124900518

In [95]:
plt.close('all')
exp.plot(rh.ts,[[rh.end.label,rh.root.label]]*len(rh.ts))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
len(angles)

18